# Part D

In this part I am going to follow same steps as part B but I am going to add two more layers to the model.

But before we move on, we have to check whether we use gpu optin in our jupyter notebook or not.(You can skip this if you just use cpu.)

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#this part is for gpu run in jupyter check. If you just use cpu based jupyter notebook, you can skip this.
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

C:\Users\leePC\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\leePC\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\leePC\anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\leePC\anaconda3\envs\tf2.0-gpu\lib\site-packages\ten

True

## Get data and Normalize it

Let's get our data and see whether we can use it or  not.

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

Just like part B, we have almost 1000 datas. We have to check whether we have null values or not.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Luckily, there are apparently no null values. So this data is ready to use.

Now, it is time to split our datas to predictors and target.

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


Since Part D does require normalize step, so I am going to normalize our data that.

In [8]:
predictors= (predictors - predictors.mean()) / predictors.std()
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Finally, we have to check our target status.

In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

Both predictors and target look fine. So we can move on to the next step.

# Import Keras and make 3 hidden layers model

As I talked before, it follows almost same steps as Part B. So I am going to quickly go through these train and just add more layers to this model.

But before we go any more further, we have to put 'sess' line since we may use more than one gpu allowed jupyter notebooks.(You can skip this if you use cpu only.)

In [11]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Using TensorFlow backend.


Compared to part B, this part D uses model with 3 hidden layers.

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    #layer 10 , relu activation
    #Add 3 hidden layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) #layer 1 => don't take this as an input layer, just putting input_shape this will be a mark for input layer.
    model.add(Dense(10, activation='relu')) #do not put input_shape in theses two layers , layer 2
    model.add(Dense(10, activation='relu')) #layer 3
    model.add(Dense(1))
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Now, we have to split our datas for getting better outcome.

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [14]:
model = regression_model()

Now, we have to train our model.

This model has 3 hideen layers(compared to Part B, Part B has one hidden layers). 

And epochs set as 50 times. (same as Part B)

In [15]:
#fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

Epoch 1/50
721/721 [==============================] - 1s 2ms/step - loss: 1517.8267
Epoch 2/50
721/721 [==============================] - 0s 131us/step - loss: 1469.8831
Epoch 3/50
721/721 [==============================] - 0s 176us/step - loss: 1402.8901
Epoch 4/50
721/721 [==============================] - 0s 134us/step - loss: 1307.4666
Epoch 5/50
721/721 [==============================] - 0s 137us/step - loss: 1173.1429
Epoch 6/50
721/721 [==============================] - 0s 178us/step - loss: 988.4741
Epoch 7/50
721/721 [==============================] - 0s 147us/step - loss: 759.1533
Epoch 8/50
721/721 [==============================] - 0s 107us/step - loss: 526.8404
Epoch 9/50
721/721 [==============================] - 0s 105us/step - loss: 361.6008
Epoch 10/50
721/721 [==============================] - 0s 161us/step - loss: 284.7124
Epoch 11/50
721/721 [==============================] - 0s 151us/step - loss: 253.1111
Epoch 12/50
721/721 [==============================] - 0s 14

131.86645213994393

## MSE compared when we add more hidden layers

Now,we have to make a list of MSE in order to compare our output.

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

131.86645462687792 0.0


We are going to print all the MSE values.
And get mean values and also standard deviation of those.

In [18]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 89.57671683351585
MSE 2: 66.05564186565313
MSE 3: 44.111832195886905
MSE 4: 46.09696269112497
MSE 5: 41.37515763711775
MSE 6: 42.542202057576105
MSE 7: 48.35149941552418
MSE 8: 37.86389379902565
MSE 9: 36.59842583122377
MSE 10: 36.77174810761387
MSE 11: 32.451714321247586
MSE 12: 30.12975689045434
MSE 13: 36.7096153493838
MSE 14: 35.21512958069835
MSE 15: 31.562215848262255
MSE 16: 22.291513122015402
MSE 17: 31.415972058441263
MSE 18: 29.086881717817683
MSE 19: 30.60241831313445
MSE 20: 28.880437029989793
MSE 21: 27.841776715127395
MSE 22: 30.26087767715207
MSE 23: 27.221111686484328
MSE 24: 26.418566367387
MSE 25: 28.746419277005984
MSE 26: 27.17182679160899
MSE 27: 25.085767968187053
MSE 28: 24.821263229962693
MSE 29: 30.907313732653375
MSE 30: 23.988339532154665
MSE 31: 22.872131193340017
MSE 32: 22.081958097933178
MSE 33: 21.505986198252458
MSE 34: 24.99024112787833
MSE 35: 24.040089962166103
MSE 36: 27.723732093391295
MSE 37: 19.867221119334395
MSE 38: 22.16980647732139
MSE

MSE in part B: Mean: 42.8598274048752

MSE in part D: Mean: 30.48035212272893

In conclusion, MSE is getting much lower(almost 15%) when we add more hidden layers into our model.

The more interesting fact is that standard deviation is also getting much lower.

In part B, standard deviation is 22.2 but in here we get a deviation of 12.2 which means the values are more centralized to the center.

However, this does not mean more hidden layers leads to betteer outcome. There is currently no theoretical reason to use neural networks with any more than two hidden layers. In fact, for many practical problems, there is no reason to use any more than one hidden layer. But for this project, more hidden layers are leads to more numbers of hidden neurons which do affect the quality of outcome.

So this is the end of our final project.

Thank you for your reading. If you have any question, please leave a comment.